In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df_train = pd.read_csv('/content/drive/MyDrive/ML/HAI817_Projet_train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/ML/HAI817_Projet_test.csv')

In [3]:
# Importation des différentes librairies, classes et fonctions utilespour le notebook

#Sickit learn met régulièrement à jour des versions et 
#indique des futurs warnings. 
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


# librairies générales
import pandas as pd
import re
from tabulate import tabulate
import time
import numpy as np
import pickle
import string
import base64
import sys

# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns

# librairies scikit learn
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


# librairies des classifiers utilisés
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

# librairies NLTK
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk import word_tokenize 

 
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
my_local_drive='/content/drive/MyDrive/ML/Prof/ML_FDS'

# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
#%cd $my_local_drive

#%pwd

# fonctions utilities (affichage, confusion, etc.)
from MyNLPUtilities import *

In [5]:
df_all = pd.concat([df_train,df_test])


In [6]:
# compter les valeurs manquantes dans chaque colonne
num_missing_values = df_all.isna().sum()
print(num_missing_values)

public_id      612
text             0
title           23
our rating       0
ID            1264
dtype: int64


**1ère tache de classification {VRAI} vs. {FAUX}**

Encodage des classes 1ère tache de classification

In [7]:
df=df_all
df_all = df_all[~df_all['our rating'].isin(['mixture', 'other'])]

df_all['classe'] = df_all['our rating'].map({'true': 1, 'false': 2})

<ipython-input-7-a826abee1647>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['classe'] = df_all['our rating'].map({'true': 1, 'false': 2})


In [8]:
print(df_all["classe"].value_counts())

2    893
1    421
Name: classe, dtype: int64


Equilibrage des classes

In [9]:
from sklearn.utils import resample
import pandas as pd



# Séparer les classes majoritaires et minoritaires
df_majority = df_all[df_all['classe'] == 2]
df_minority = df_all[df_all['classe'] == 1]

# Sous-échantillonner la classe majoritaire
df_majority_downsampled = resample(df_majority, 
                                   replace=False,    # Échantillonnage sans remplacement
                                   n_samples=len(df_minority), # Nombre d'échantillons égal à la classe minoritaire
                                   random_state=42)  # Pour la reproductibilité

# Combiner les classes majoritaire et minoritaire
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Afficher la taille du jeu de données équilibré
print("Taille du jeu de données équilibré : ", df_balanced.shape)

df_all = df_balanced
print(df_all['classe'].value_counts())

Taille du jeu de données équilibré :  (842, 6)
2    421
1    421
Name: classe, dtype: int64


Définition de la fonction  MyCleanText

In [10]:
import re
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

def MyCleanText(X,
 lowercase=False, # mettre en minuscule
 removestopwords=False, # supprimer les stopwords
 removedigit=False, # supprimer les nombres
 getstemmer=False, # conserver la racine des termes
 getlemmatisation=False # lematisation des termes
 ):
    
    sentence = str(X)
    
    # suppression des caractères spéciaux
    sentence = re.sub(r'[^\w\s]',' ', sentence)
    
    # suppression de tous les caractères uniques
    sentence = re.sub(r'\s+[a-zA-Z]\s+', ' ', sentence)
    
    # substitution des espaces multiples par un seul espace
    sentence = re.sub(r'\s+', ' ', sentence, flags=re.I)
    
    # decoupage en mots
    tokens = word_tokenize(sentence)
    
    if lowercase:
        tokens = [token.lower() for token in tokens]

    # suppression ponctuation
    table = str.maketrans('', '', string.punctuation)
    words = [token.translate(table) for token in tokens]
    
    # suppression des tokens non alphabetique ou numerique
    words = [word for word in words if word.isalnum()]

    # suppression des tokens numerique
    if removedigit:
        words = [word for word in words if not word.isdigit()]
    
    # suppression des stopwords
    if removestopwords:
        words = [word for word in words if not word in stop_words]
    
    # lemmatisation
    if getlemmatisation:
        lemmatizer=WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word)for word in words]
    
    # racinisation
    if getstemmer:
        ps = PorterStemmer()
        words=[ps.stem(word) for word in words]

    sentence= ' '.join(words)

    return sentence


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Définition de la fonction TextNormalizer

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin
class TextNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self,
                 removestopwords=False, # suppression des stopwords
                 lowercase=False,# passage en minuscule
                 removedigit=False, # supprimer les nombres
                 getstemmer=False,# racinisation des termes
                 getlemmatisation=False # lemmatisation des termes
                ):
        self.lowercase=lowercase
        self.getstemmer=getstemmer
        self.removestopwords=removestopwords
        self.getlemmatisation=getlemmatisation
        self.removedigit=removedigit
    
    def transform(self, X, **transform_params):
        # Nettoyage du texte
        X=X.copy() # pour conserver le fichier d'origine
        return [MyCleanText(text, lowercase=self.lowercase,
                            getstemmer=self.getstemmer,
                            removestopwords=self.removestopwords,
                            getlemmatisation=self.getlemmatisation,
                            removedigit=self.removedigit) for text in X]
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X).transform(X)
    
    def get_params(self, deep=True):
        return {
            'lowercase':self.lowercase,
            'getstemmer':self.getstemmer,
            'removestopwords':self.removestopwords,
            'getlemmatisation':self.getlemmatisation,
            'removedigit':self.removedigit
        }
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self


Pipeline de classification de texte avec sélection de features discriminantes

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline



# Séparation des données en ensembles d'entraînement et de test
X = df_all['text']
y = df_all['classe']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Initialisation du pipeline
pipe = Pipeline([
    ("cleaner", TextNormalizer(removestopwords=True, lowercase=True, removedigit=True, getstemmer=False, getlemmatisation=False)),
    ("TfidfVectorizer", TfidfVectorizer()), 
    ("selector", SelectKBest(chi2, k=int(len(X_train)*0.1))),

    ("svm", SVC(C=1, gamma=0.001, kernel='linear'))
])




# Ajustement du pipeline sur les données d'entraînement
pipe.fit(X_train, y_train)

# Utilisation de SelectKBest pour sélectionner les 10% des features les plus discriminantes

selector= SelectKBest(chi2, k=int(len(X_train)*0.1))




# Récupération de la liste des features discriminantes
feature_names = list(pipe.named_steps['TfidfVectorizer'].vocabulary_.keys())
mask = pipe.named_steps['selector'].get_support()
selected_features = [feature_names[i] for i, val in enumerate(mask) if val]

# Affichage des features sélectionnées
print("Liste des features discriminantes :")
for feature in selected_features:
    print(feature)


Liste des features discriminantes :
inspectors
hipster
wedding
captain
specifically
miami
alleged
politico
failed
disapproved
js
consumed
asteroidwatch
difficulties
unable
redeem
identification
code
withdrew
owning
accepted
wigdortz
floating
transantarctic
generates
anywhere
vdawtd
resignation
shuttered
meant
tossed
introducing
township
kansas
israelis
dealers
rosemary
assesses
leo
unbound
coaches
embargoes
roberta
cmvdh9rpgl
pi
nazis
reliefs
limousine
weighs
volatile
neckline
pasteur
lawmaker
mystic
con
shippers
alok
que
yenne
drumming
alt
widest
overused
montagnier
cajeput
emblematic
dropout


**2ème tache de classification {VRAI ou FAUX} vs. {AUTRE}**

In [13]:
import pandas as pd


df2=df
# Encoder les classes en deux catégories : "VRAI ou FAUX" et "AUTRE"
df['classe'] = df['our rating'].replace(['mixture', 'other'], 'AUTRE')
df['classe'] = df['classe'].replace(['true', 'false'], 'TrueFalse')


df['classe'] = df['classe'].map({'TrueFalse': 1, 'AUTRE': 2})

from sklearn.utils import resample
import pandas as pd



# Séparer les classes majoritaires et minoritaires
df_majority = df[df['classe'] == 1]
df_minority = df[df['classe'] == 2]

# Sous-échantillonner la classe majoritaire
df_majority_downsampled = resample(df_majority, 
                                   replace=False,    # Échantillonnage sans remplacement
                                   n_samples=len(df_minority), # Nombre d'échantillons égal à la classe minoritaire
                                   random_state=42)  # Pour la reproductibilité

# Combiner les classes majoritaire et minoritaire
df_balanced = pd.concat([df_majority_downsampled, df_minority])



df = df_balanced
print(df['classe'].value_counts())

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline



# Séparation des données en ensembles d'entraînement et de test
X = df['text']
y = df['classe']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Initialisation du pipeline
pipe = Pipeline([
    ("cleaner", TextNormalizer(removestopwords=True, lowercase=True, removedigit=True, getstemmer=False, getlemmatisation=False)),
    ("TfidfVectorizer", TfidfVectorizer()), 
    ("selector", SelectKBest(chi2, k=int(len(X_train)*0.1))),

    ("svm", SVC(C=1, gamma=0.001, kernel='linear'))
])




# Ajustement du pipeline sur les données d'entraînement
pipe.fit(X_train, y_train)

# Utilisation de SelectKBest pour sélectionner les 10% des features les plus discriminantes

selector= SelectKBest(chi2, k=int(len(X_train)*0.1))




# Récupération de la liste des features discriminantes
feature_names = list(pipe.named_steps['TfidfVectorizer'].vocabulary_.keys())
mask = pipe.named_steps['selector'].get_support()
selected_features = [feature_names[i] for i, val in enumerate(mask) if val]

# Affichage des features sélectionnées
print("Liste des features discriminantes :")
for feature in selected_features:
    print(feature)



1    562
2    562
Name: classe, dtype: int64
Liste des features discriminantes :
related
goal
officials
river
live
contaminated
genuine
danish
bully
proponents
socioeconomic
respectively
broke
capped
officially
coronaviruses
cybercriminals
cybercrime
toxic
homeless
structures
deployed
strategies
tropics
deceased
meetings
dominic
tighten
pains
marshallese
battling
preexisting
incarnate
unfathomably
tnt
pml
individualism
harwood
donates
zoologist
transcribes
attract
carswell
pandemics
hodeida
vanishingly
treading
enablers
valter
coworkers
salvadoran
javier
payroll
positively
rulers
kensington
apprehending
rushing
pitch
reims
aahs
shanxi
biomedical
repelled
firefighter
interfered
faire
demoralized
odegbune
sleek
quantitative
mercedes
fermi
compresses
dayside
riddle
merging
wilshaw
julian
25nmol
unloaded
roaring
rebadging
5e
consol
sapphire
vogue
sunglasses
shortcuts


**3ème tache de classification {VRAI} vs. {FAUX} vs. {MIXTE} vs. {AUTRE}**

In [14]:


df2['classe'] = df2['our rating'].map({'true': 1, 'false': 2, 'mixture': 3, 'other': 4})


# Downsampling de la classe majoritaire
false_downsampled = resample(df2[df2['our rating'] == 'false'], replace=False, n_samples=250, random_state=42)

# Upsampling de la classe minoritaire
true_upsampled = resample(df2[df2['our rating'] == 'true'], replace=False, n_samples=250, random_state=42)
mixture_upsampled = resample(df2[df2['our rating'] == 'mixture'], replace=False, n_samples=250, random_state=42)
other_upsampled = resample(df2[df2['our rating'] == 'other'], replace=True, n_samples=250, random_state=42)

# Concaténer les données échantillonnées
balanced_data = pd.concat([false_downsampled, true_upsampled, mixture_upsampled, other_upsampled])

# Afficher les nouvelles proportions des classes
df2 = balanced_data

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline



# Séparation des données en ensembles d'entraînement et de test
X = df2['text']
y = df2['classe']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Initialisation du pipeline
pipe = Pipeline([
    ("cleaner", TextNormalizer(removestopwords=True, lowercase=True, removedigit=True, getstemmer=False, getlemmatisation=False)),
    ("TfidfVectorizer", TfidfVectorizer()), 
    ("selector", SelectKBest(chi2, k=int(len(X_train)*0.1))),

    ("svm", SVC(C=1, gamma=0.001, kernel='linear'))
])




# Ajustement du pipeline sur les données d'entraînement
pipe.fit(X_train, y_train)

# Utilisation de SelectKBest pour sélectionner les 10% des features les plus discriminantes

selector= SelectKBest(chi2, k=int(len(X_train)*0.1))




# Récupération de la liste des features discriminantes
feature_names = list(pipe.named_steps['TfidfVectorizer'].vocabulary_.keys())
mask = pipe.named_steps['selector'].get_support()
selected_features = [feature_names[i] for i, val in enumerate(mask) if val]

# Affichage des features sélectionnées
print("Liste des features discriminantes :")
for feature in selected_features:
    print(feature)



Liste des features discriminantes :
negotiating
society
flgov
debris
removing
county
dominion
photo
pregnancies
win
intended
misleadingly
planning
smaller
newspaper
terminated
john
injuries
imposing
prices
mohammed
tour
partly
icebridge
grindhouse
interactive
sturgeon
salinity
rahmstorf
csl
subtleties
orwellian
flourish
listened
notable
zimbabwe
object
bonn
showcasing
flynn
yelled
forbidding
fainted
korean
jakobshavn
conditioner
lifesitenews
ovulation
messaged
bandana
convenience
palliative
injure
negotiation
ahold
sicker
daddy
cancun
shah
physiology
streaming
hutchinson
golfers
nanoscience
gill
psn
contended
attain
sceptic
mao
arlene
mirna
snaking
dad
ealing
liveability
robbed
andymurray
ponzi
millimeters
